## Applied Data Science Capstone Week 5
Segmenting and Clustering Neighborhoods in Toronto by venue and crime
Jim Foster

In [2]:
import pandas as pd
import numpy as np

toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')# source of scrape data from Wikipedia
toronto_neighborhood = toronto[0] # this is the table we need
print(toronto_neighborhood.shape) # understand the size of the data set
toronto_neighborhood.head() # visual inspection of some of the data set

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
toronto_neighborhood_filtered = toronto_neighborhood[toronto_neighborhood['Borough']!= 'Not assigned']
# removed rows where Boroughs = 'not assigned' from the data set

In [4]:
print(toronto_neighborhood_filtered.shape)# understand how the shape has changed
toronto_neighborhood_filtered.head()

(103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
toronto_neighborhood_filtered = toronto_neighborhood_filtered.reset_index(drop=True) # reset the index for the file
print(toronto_neighborhood_filtered.shape) # verify that have not lost any rows
toronto_neighborhood_filtered

(103, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
toronto_neighborhood_filtered.shape

(103, 3)

Part 2 Get geocoding information

In [7]:
can_post_code = pd.read_csv(r'http://cocl.us/Geospatial_data') 
can_post_code.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
toronto_neighborhood_longlat = toronto_neighborhood_filtered.set_index('Postal Code').join(can_post_code.set_index('Postal Code'))


In [9]:
print(toronto_neighborhood_longlat.shape)
toronto_neighborhood_longlat.head()

(103, 4)


,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
toronto_neighborhood_longlat.reset_index(inplace=True)
print(toronto_neighborhood_longlat.shape)
toronto_neighborhood_longlat.head()

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Exploring the Toronto Neighborhoods

In [10]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [12]:
CLIENT_ID = 'D4ZADADFF02W5ZPF4YOYEFTUTTKRC2VCIREZ21V0EXTSFYH1' # your Foursquare ID
CLIENT_SECRET = 'EXCGOPAMV3TFKYGCPYQITGRG1DEHZLTUQA5BN0UJYJPYH0WM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D4ZADADFF02W5ZPF4YOYEFTUTTKRC2VCIREZ21V0EXTSFYH1
CLIENT_SECRET:EXCGOPAMV3TFKYGCPYQITGRG1DEHZLTUQA5BN0UJYJPYH0WM


In [16]:
toronto_neighborhood_longlat.loc[0,'Neighbourhood'] # peak at the data

'Parkwoods'

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Toronto_venues =getNearbyVenues(names=toronto_neighborhood_longlat['Neighbourhood'],latitudes=toronto_neighborhood_longlat['Latitude'],longitudes=toronto_neighborhood_longlat['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2141, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [21]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


In [22]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


In [23]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [24]:
Toronto_onehot.shape # understand the shape of the data

(2141, 273)

In [25]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped

(96, 273)


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,

In [26]:
Toronto_crime = pd.read_csv('C:\projects\Toronto_Crime_Data_2019_Neighborhood.csv') # read in the crime data by Postal Code
Toronto_crime.head()

,Neighbourhood,Postal Code,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Latitude,Longitude
0,Malvern,M1B,333,71,44,46,10,43.804568,-79.221024
1,Rouge,M1B,182,87,59,32,13,43.810563,-79.189389
2,Centennial Scarborough,M1C,48,19,29,1,3,43.783578,-79.151530
3,Highland Creek,M1C,58,27,33,9,1,43.790714,-79.174422
4,Guildwood,M1E,28,3,10,7,1,43.747495,-79.201061


In [27]:
Toronto_interim =pd.merge(toronto_neighborhood_longlat, Toronto_crime, on = "Postal Code")# add postal codes to Toronto_grouped
Toronto_combined = Toronto_interim.drop(['Neighbourhood_y','Latitude_y','Longitude_y','Borough','Postal Code','Latitude_x','Longitude_x'], axis=1) 
# get rid of extranous columns
Toronto_combined.rename(columns = {"Neighbourhood_x": "Neighborhood"}, inplace = True)
#Toronto_combined.rename(columns = {"Latitude_x": "Latitude"}, inplace = True)
#Toronto_combined.rename(columns = {"Longitude_x": "Longitude"}, inplace = True)
print(Toronto_combined.shape)
Toronto_combined.head()


(138, 6)


,Neighborhood,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,Parkwoods,158,32,68,36,5
1,"Regent Park, Harbourfront",156,21,65,14,5
2,"Lawrence Manor, Lawrence Heights",209,61,84,42,29
3,"Queen's Park, Ontario Provincial Government",124,9,91,12,6
4,"Islington Avenue, Humber Valley Village",33,23,54,5,8


In [28]:
Toronto_combined['Assault'] = (Toronto_combined['Assault']/Toronto_combined['Assault'].sum())#change crimes to percentages
Toronto_combined['Auto Theft'] = (Toronto_combined['Auto Theft']/Toronto_combined['Auto Theft'].sum())#change crimes to percentages
Toronto_combined['Break and Enter'] = (Toronto_combined['Break and Enter']/Toronto_combined['Break and Enter'].sum())#change crimes to percentages
Toronto_combined['Robbery'] = (Toronto_combined['Robbery']/Toronto_combined['Robbery'].sum())#change crimes to percentages
Toronto_combined['Theft Over'] = (Toronto_combined['Theft Over']/Toronto_combined['Theft Over'].sum())#change crimes to percentages

In [29]:
Toronto_combined.head()

,Neighborhood,Assault,Auto Theft,Break and Enter,Robbery,Theft Over
0,Parkwoods,0.007814,0.006196,0.008194,0.010594,0.003717
1,"Regent Park, Harbourfront",0.007715,0.004066,0.007832,0.004120,0.003717
2,"Lawrence Manor, Lawrence Heights",0.010336,0.011810,0.010122,0.012360,0.021561
3,"Queen's Park, Ontario Provincial Government",0.006132,0.001742,0.010965,0.003531,0.004461
4,"Islington Avenue, Humber Valley Village",0.001632,0.004453,0.006507,0.001471,0.005948


In [30]:
Toronto_total =pd.merge(Toronto_combined, Toronto_grouped, on = "Neighborhood")
print(Toronto_total.shape)
Toronto_total.head()

(135, 278)


,Neighborhood,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Parkwoods,0.007814,0.00

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
# understand what is making each of the Neighborhoods unique by looking at what people are doing
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_total['Neighborhood']

for ind in np.arange(Toronto_total.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_total.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Robbery,Break and Enter,Assault,Auto Theft,Theft Over,Distribution Center,Department Store,Dessert Shop
1,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Dessert Shop,Art Gallery,French Restaurant
2,"Lawrence Manor, Lawrence Heights",Clothing Store,Women's Store,Gift Shop,Vietnamese Restaurant,Accessories Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Theft Over
3,"Queen's Park, Ontario Provincial Government",Coffee Shop,Creperie,Beer Bar,Bar,Fried Chicken Joint,Mexican Restaurant,Park,Sushi Restaurant,Hobby Shop,Art Gallery
4,"Malvern, Rouge",Fast Food Restaurant,Assault,Auto Theft,Robbery,Theft Over,Break and Enter,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner


In [33]:
Toronto_grouped_clustering = Toronto_total.drop(['Neighborhood'], axis = 1)
Toronto_grouped_clustering.head()

,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0.007814,0.006196,0.008194,0.010594,

In [34]:
# set number of clusters
kclusters = 7

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 2, 2, 3, 3, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 4, 6, 6, 6,
       6, 0, 0, 6, 6, 6, 6, 6, 6, 6, 5, 5, 6, 6, 6, 0, 0, 6, 0, 0, 1, 1,
       1, 6, 6, 1, 6, 0, 6, 6, 6, 6, 6, 6])

In [35]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = toronto_neighborhood_longlat

# merge Toronto_merged with Toronto areas and add Latitude and Longitude
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
print(Toronto_merged.shape)
Toronto_merged.head() # check to make sure cluster labels are included

(159, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Robbery,Break and Enter,Assault,Auto Theft,Theft Over,Distribution Center,Department Store,Dessert Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6.0,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Dessert Shop,Art Gallery,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6.0,Clothing Store,Women's Store,Gift Shop,Vietnamese Restaurant,Accessories Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Theft Over
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,6.0,Coffee Shop,Creperie,Beer Bar,Bar,Fried Chicken Joint,Mexican Restaurant,Park,Sushi Restaurant,Hobby Shop,Art Gallery


In [36]:
print(Toronto_merged.shape) # In
Toronto_merged =Toronto_merged.dropna() # Drop rows with "NaN"
print(Toronto_merged.shape)
Toronto_merged.head()

(159, 16)
(143, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Robbery,Break and Enter,Assault,Auto Theft,Theft Over,Distribution Center,Department Store,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6.0,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Dessert Shop,Art Gallery,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6.0,Clothing Store,Women's Store,Gift Shop,Vietnamese Restaurant,Accessories Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Theft Over
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,6.0,Coffee Shop,Creperie,Beer Bar,Bar,Fried Chicken Joint,Mexican Restaurant,Park,Sushi Restaurant,Hobby Shop,Art Gallery
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,6.0,Fast Food Restaurant,Assault,Auto Theft,Robbery,Theft Over,Break and Enter,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner


In [37]:
Toronto_merged['Cluster Labels'].value_counts() # look at the distribution of clusters. 

6.0    108
0.0     19
1.0      8
3.0      3
5.0      2
2.0      2
4.0      1
Name: Cluster Labels, dtype: int64

In [38]:
from geopy.geocoders import Nominatim # import basis for maps

In [39]:
address = 'Toronto,Ontario' # get center of map for folium

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [40]:
Toronto_merged["Cluster Labels"] = Toronto_merged["Cluster Labels"].replace(np.nan, 0) #Changing any "NaN" to 0
Toronto_merged["Cluster Labels"] = Toronto_merged["Cluster Labels"].astype(int)
Toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Food & Drink Shop,Robbery,Break and Enter,Assault,Auto Theft,Theft Over,Distribution Center,Department Store,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,6,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Dessert Shop,Art Gallery,French Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,6,Clothing Store,Women's Store,Gift Shop,Vietnamese Restaurant,Accessories Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Theft Over
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,6,Coffee Shop,Creperie,Beer Bar,Bar,Fried Chicken Joint,Mexican Restaurant,Park,Sushi Restaurant,Hobby Shop,Art Gallery
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,6,Fast Food Restaurant,Assault,Auto Theft,Robbery,Theft Over,Break and Enter,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,North York,0,Pub,Park,Japanese Restaurant,Bakery,Pizza Place,Robbery,Assault,Auto Theft,Theft Over,Break and Enter
10,North York,0,Pub,Park,Japanese Restaurant,Bakery,Pizza Place,Assault,Theft Over,Robbery,Auto Theft,Break and Enter
35,East York,0,Park,Intersection,Convenience Store,Assault,Theft Over,Robbery,Break and Enter,Auto Theft,Doner Restaurant,Dim Sum Restaurant
35,East York,0,Park,Intersection,Convenience Store,Assault,Break and Enter,Robbery,Theft Over,Auto Theft,College Rec Center,Dessert Shop
35,East York,0,Park,Convenience Store,Intersection,Robbery,Assault,Theft Over,Break and Enter,Auto Theft,Distribution Center,Dessert Shop
44,Scarborough,0,Bakery,Bus Line,Ice Cream Shop,Park,Metro Station,Bus Station,Intersection,Soccer Field,Assault,Auto Theft
44,Scarborough,0,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Bus Station,Intersection,Park,Robbery,Assault
49,North York,0,Park,Bakery,Construction & Landscaping,Theft Over,Auto Theft,Break and Enter,Robbery,Assault,Gourmet Shop,Golf Course
49,North York,0,Park,Bakery,Construction & Landscaping,Auto Theft,Theft Over,Break and Enter,Assault,Robbery,Greek Restaurant,Deli / Bodega
61,Central Toronto,0,Park,Swim School,Bus Line,Auto Theft,Break and Enter,Assault,Theft Over,Robbery,Distribution Center,Dessert Shop


In [44]:
Cluster_0 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_0['Latitude'], Cluster_0['Longitude'], Cluster_0['Neighbourhood'],Cluster_0['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="Red",
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [116]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Park,Food & Drink Shop,Robbery,Break and Enter,Assault,Auto Theft,Theft Over,Distribution Center,Department Store,Dessert Shop
21,York,1,Park,Women's Store,Pool,Robbery,Break and Enter,Theft Over,Assault,Auto Theft,Turkish Restaurant,Discount Store
21,York,1,Park,Women's Store,Pool,Robbery,Assault,Auto Theft,Theft Over,Break and Enter,Video Game Store,Dog Run
64,York,1,Park,Auto Theft,Assault,Break and Enter,Robbery,Theft Over,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant
64,York,1,Park,Assault,Auto Theft,Robbery,Break and Enter,Theft Over,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant
64,York,1,Park,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant
66,North York,1,Park,Convenience Store,Break and Enter,Auto Theft,Theft Over,Robbery,Assault,Greek Restaurant,Escape Room,Eastern European Restaurant
91,Downtown Toronto,1,Park,Trail,Playground,Break and Enter,Theft Over,Assault,Robbery,Auto Theft,Dog Run,Department Store


In [45]:
Cluster_1 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_1['Latitude'], Cluster_1['Longitude'], Cluster_1['Neighbourhood'],Cluster_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="Blue",
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [118]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,2,Print Shop,Auto Theft,Robbery,Break and Enter,Theft Over,Assault,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
11,Etobicoke,2,Print Shop,Auto Theft,Theft Over,Break and Enter,Assault,Robbery,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner


In [46]:
Cluster_2 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_2['Latitude'], Cluster_2['Longitude'], Cluster_2['Neighbourhood'], Cluster_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="Green",
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [120]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,3,Bar,Construction & Landscaping,Auto Theft,Break and Enter,Assault,Theft Over,Robbery,Diner,Discount Store,Distribution Center
12,Scarborough,3,Bar,Construction & Landscaping,Auto Theft,Break and Enter,Assault,Robbery,Theft Over,Diner,Discount Store,Distribution Center
101,Etobicoke,3,Construction & Landscaping,Baseball Field,Break and Enter,Auto Theft,Robbery,Assault,Theft Over,Donut Shop,Diner,Discount Store


In [47]:
Cluster_3 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_3['Latitude'], Cluster_3['Longitude'], Cluster_3['Neighbourhood'],Cluster_3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-4],
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [122]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4,Martial Arts School,Break and Enter,Robbery,Theft Over,Assault,Auto Theft,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [123]:
Cluster_4 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_4['Latitude'], Cluster_4['Longitude'], Cluster_4['Neighbourhood'],Cluster_4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-4],
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [124]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,5,Baseball Field,Auto Theft,Assault,Robbery,Break and Enter,Theft Over,Donut Shop,Dim Sum Restaurant,Diner,Discount Store
57,North York,5,Baseball Field,Auto Theft,Theft Over,Assault,Break and Enter,Robbery,Donut Shop,Dim Sum Restaurant,Diner,Discount Store


In [125]:
Cluster_5 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5]
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_5['Latitude'], Cluster_5['Longitude'], Cluster_5['Neighbourhood'],Cluster_5['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-4],
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [126]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,6,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Dessert Shop,Art Gallery,French Restaurant
3,North York,6,Clothing Store,Women's Store,Gift Shop,Vietnamese Restaurant,Accessories Store,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Theft Over
4,Downtown Toronto,6,Coffee Shop,Creperie,Beer Bar,Bar,Fried Chicken Joint,Mexican Restaurant,Park,Sushi Restaurant,Hobby Shop,Art Gallery
6,Scarborough,6,Fast Food Restaurant,Assault,Auto Theft,Robbery,Theft Over,Break and Enter,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
6,Scarborough,6,Fast Food Restaurant,Auto Theft,Theft Over,Robbery,Assault,Break and Enter,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant
7,North York,6,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Discount Store,Sporting Goods Shop,Art Gallery,Italian Restaurant,Bike Shop,Sandwich Place
7,North York,6,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Restaurant,Supermarket,Sporting Goods Shop,Italian Restaurant,Bike Shop,Sandwich Place
8,East York,6,Pizza Place,Pharmacy,Pet Store,Intersection,Bank,Athletics & Sports,Bus Line,Gym / Fitness Center,Breakfast Spot,Gastropub
9,Downtown Toronto,6,Clothing Store,Coffee Shop,Robbery,Café,Assault,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Break and Enter,Hotel
13,North York,6,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Discount Store,Sporting Goods Shop,Art Gallery,Italian Restaurant,Bike Shop,Sandwich Place


In [127]:
Cluster_6 = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6]

In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_6['Latitude'], Cluster_6['Longitude'], Cluster_6['Neighbourhood'],Cluster_6['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-4],
        fill=True,
        fill_color=rainbow[cluster-4],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters